# Sprint17 Faster R-CNNとyolo-V3

# yolo-V3

## プログラム準備
https://github.com/qqwweee/keras-yolo3

上記で取得してきたフォルダの中に、このSprint17-yolo.ipynbを配置します。

## データ準備
faster r cnn で取得してきたデータをこのSprint17-yolo.ipynbと同階層に設置します。

## 学習済みモデルの取得

下記にアクセスし、学習済みモデルを取得してきます。

https://pjreddie.com/media/files/yolov3.weights

## データ確認

このSprint17-yolo.ipynbがあるフォルダ内は下記のようになっていると思います。

- 上記リポジトリのファイルやディレクトリ群（ファイルやディレクトリ）
- kaggle_simpson_testset /（ディレクトリ）
- simpsons_dataset /（ディレクトリ）
- Sprint17-rcnn.ipynb（ファイル）
- yolov3.weights

## モデル形式の変更

ここで転移学習を行う。  
ネットから拾ってきた重みをかきこんている

In [1]:
!python convert.py yolov3.cfg yolov3.weights model_data/yolo_weights.h5

Using TensorFlow backend.
Loading weights.
Weights Header:  0 2 0 [32013312]
Parsing Darknet config.
Creating Keras model.


Parsing section net_0
Parsing section convolutional_0
conv2d bn leaky (3, 3, 3, 32)




2021-09-03 16:12:40.303573: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-09-03 16:12:40.344672: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fb6157b3120 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-09-03 16:12:40.344710: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version





Parsing section convolutional_1
conv2d bn leaky (3, 3, 32, 64)
Parsing section convolutional_2
conv2d bn leaky (1, 1, 64, 32)
Parsing section convolutional_3
conv2d bn leaky (3, 3, 32, 64)
Parsing section shortcut_0
Parsing section convolutional_4
conv2d bn leaky (3, 3, 64, 128)
Parsin

# 問題3　学習済みの重みによる推定

上で引っ張ってきた重みを使って適当な画像を分類している。  
クラスはあらかじめ用意されていたbirdを想定する。  
ここではアノテーションファイルはない

In [5]:
python yolo_video.py --image

Using TensorFlow backend.
Image detection mode
 Ignoring remaining command line arguments: ./path2your_video,



2021-09-03 16:25:08.691808: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-09-03 16:25:08.701965: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fc0e5325100 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-09-03 16:25:08.701982: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version









model_data/yolo.h5 model, anchors, and classes loaded.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Input image filename:^C
Traceback (most recent call last):
  File "yolo_video.py", line 73, in <module>
    detect_img(YOLO(**vars(FLAGS)))
  File "yolo_video.py", line 8, in detect_img
    img = input('Input image filename:')
Keyboa

ターミナルで実行
![Imgur](https://i.imgur.com/EyFTGiJ.png)

# 問題4　学習のためのファイルを作成

annotation.txtのファイル形式がfaster-rcnnの形式と違うため、変更したものを作成する

rcnnの際に使ったannotation.txtをこのフォルダにコピーし、annotation_rcnn.txtとrenameしてください。

このアノテーションはシンプソンのクラス分けにしか使えないアノテーションになる。  
アノテーション自体は理解できたが、クラスがないのでラベリングしている？？

In [6]:
# ライブラリのimport
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [7]:
# 読み込みと確認
annotation_df = pd.read_csv('annotation_rcnn.txt',header=None)
n_sample, n_col = annotation_df.shape
annotation_df.head()

,0,1,2,3,4,5
0,simpsons_dataset/abraham_grampa_simpson/pic_00...,57,72,52,72,abraham_grampa_simpson
1,simpsons_dataset/abraham_grampa_simpson/pic_00...,80,31,337,354,abraham_grampa_simpson
2,simpsons_dataset/abraham_grampa_simpson/pic_00...,128,48,285,407,abraham_grampa_simpson
3,simpsons_dataset/abraham_grampa_simpson/pic_00...,72,126,158,275,abraham_grampa_simpson
4,simpsons_dataset/abraham_grampa_simpson/pic_00...,123,61,294,416,abraham_grampa_simpson


In [8]:
# キャラクター名を数値（クラス・ラベル）に変更する
le = LabelEncoder()
annotation_df.iloc[:,5] = le.fit_transform(annotation_df.iloc[:,5])
annotation_df.head()

,0,1,2,3,4,5
0,simpsons_dataset/abraham_grampa_simpson/pic_00...,57,72,52,72,0
1,simpsons_dataset/abraham_grampa_simpson/pic_00...,80,31,337,354,0
2,simpsons_dataset/abraham_grampa_simpson/pic_00...,128,48,285,407,0
3,simpsons_dataset/abraham_grampa_simpson/pic_00...,72,126,158,275,0
4,simpsons_dataset/abraham_grampa_simpson/pic_00...,123,61,294,416,0


In [9]:
rcnn_path = 'annotation_rcnn.txt'
yolo_path = 'annotation.txt'

# rcnnの方のannotation.txtを開く
with open(rcnn_path) as f:
    # データを読み込む
    lines = f.readline() 
    # データ数（行数）でループ
    for i in range(n_sample):
        # カンマ区切りのデータをリストに変換
        split_line = lines.split(',') 
        # 画像のパスは一番前に格納されているので取得
        image_path = split_line[0]
        # パス変更
        split_line[0] = './' + image_path
        # 末尾をキャラクター名から数値（クラス・ラベル）に変更→改行コード加える
        split_line[-1] = str(annotation_df.iloc[i,5]) + '\n'  
        # yoloの方のannotation.txtを開き保存
        with open(yolo_path, mode='a') as out_f:
            join_line = ','.join(split_line)  
            join_line = join_line.replace('.jpg,','.jpg ')  
            out_f.write(join_line)

# 問題5　学習が行えることの確認


問題4で作成したファイルを使用して学習してください。実行環境で学習に時間がかかる場合は、学習が行えることを確認するのみで終えて構いません。

In [12]:
!python train.py

Using TensorFlow backend.








2021-09-03 16:40:55.964389: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-09-03 16:40:55.975264: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7ff3aef7d980 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-09-03 16:40:55.975283: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version





Create YOLOv3 model with 9 anchors and 20 classes.
/opt/anaconda3/envs/sinpson/lib/python3.6/site-packages/keras/engine/saving.py:1140: UserWarning: Skipping loading of weights for layer conv2d_59 due to mismatch in shape ((1, 1, 1024, 75) vs (255, 1024, 1, 1)).
  weight_values[i].shape))
/opt/anaconda3/envs/sinpson/lib/python3.6/site-packages/keras/engine/saving.py:1140: UserWarning: Skipping loading of weights for layer conv2d_59 due to mismatch in shape ((75

# 考察

今回の学習を行う中での疑問

1. 任意画像の分類後のアノテーションファイルの書き出しについて  
    分別させた画像のアノテーションファイルがどこに書き出されているのか不明であった　。  
    このデータがあれば、データ拡張をする際バウンディボックスを作成するのが楽になると考えられる。  


2. シンプソンのクラスについて  
    今回シンプソンのアノテーションファイルを書き換えて、最後にラベルのデータがくる様にしたがそのままではダメなのだろうか？  
    voc_annotation.pyのクラスリストを書き換え、それに対応するindexを出力すればシンプソンの名前でクラスわけされるのではないだろうか。  


3. 最終的にシンプソンのアノテーションを学習させ終わった後では、シンプソンの顔以外全く判断できないモデルになるのだろうか。